# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import streamlit as st
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import requests
from io import StringIO
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from urllib.parse import urlencode

# Uncomment when ready for machine learning:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Pulling up our .env file:
load_dotenv()

True

# Data Preprocessing

## Object Declarations and Functions

In [4]:
# Function Sets and Parameters

function_set = ['TIME_SERIES_DAILY', 'RSI', 'NEWS_SENTIMENT', 'MACD']
single_function = ['NEWS_SENTIMENT']
sa_function_parameters = ['technology', 'retail_wholesale']
premium_function_set = ['MACD']

# A list of functions featured in the Alpha Vantage API and their parameters:
parameters = {
    'TIME_SERIES_DAILY': {
        'function': 'TIME_SERIES_DAILY',
        'symbol': None,
        'outputsize': 'full',
        'datatype': None,
        'apikey': None
        },
    'NEWS_SENTIMENT': {
        'function': 'NEWS_SENTIMENT',
        'tickers': None,
        'limit': 1000,
        'apikey': None
        },
    'RSI': {
        'function':'RSI',
        'symbol': None,
        'interval': 'daily',
        'time_period': 14,
        'series_type': 'close',
        'datatype': None,
        'apikey': None
        },
    'REAL_GDP': {
        'function': 'REAL_GDP',
        'interval':'quarterly',
        'datatype': None,
        'apikey': None
        },
    'MACD': {
        'function': 'MACD',
        'symbol': None,
        'interval': 'daily',
        'series_type':'close',
        'datatype': None,
        'apikey': None
    }
}

In [15]:
# Tracking (ONLY RUN THIS CELL ONCE PER SESSION)

session_calls = 0

In [6]:
# Data Prep Function
#------
# Extracts a given DataFrame to a data folder within the working directory, appending a date value to the name.
#------

In [7]:
# Export CSV Function
#------
# Extracts a given DataFrame to a data folder within the working directory, appending a date value to the name.
#------

def export_csv(df, file_name: str):

    # Conditional to verify that "file_name" is a string:
    if not isinstance(file_name, str):
        raise TypeError(f'The "file_name" parameter must be a string; it\'s currently {str(type(file_name)).upper()}.')

    # Declares the current date:
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Creates the data folder within the current working directory, utilizing a Path object that pulls the current working directory:
    path = Path('{}/data'.format(os.getcwd()))
    path.mkdir(parents=True, exist_ok=True)
    extract_to = '{}/{}_{}.csv'.format(path, current_date, file_name)

    # Exports the CSV file:
    df.to_csv(extract_to)

    return print('Exported your DataFrame to \'{}\'!'.format(extract_to))

In [8]:
# Generate Features Function
#------
# Generates different lagged close, rolling mean/std and relative change features for the daily time series data.
#------

def generate_features(df):

    # Declaring the location of the close column; to insert the new features next it:
    close_location = df.columns.get_loc('close')

    # Lagged Close
    df.insert(close_location + 1, 'lag_1', df['close'].shift(1))
    df.insert(close_location + 2, 'lag_2', df['close'].shift(2))

    # Windowed Mean and STD
    df.insert(close_location + 3, 'rolling_mean_7', df['close'].rolling(window=7).mean())
    df.insert(close_location + 4, 'rolling_std_7', df['close'].rolling(window=7).std())

    # Relative Change
    df.insert(close_location + 5, 'daily_return', df['close'].pct_change() * 100)

    # Drops any rows with resulting null values:
    df.dropna(inplace=True)

    return df

In [9]:
# Sentiment Extraction Function
#------
# Applied to the resulting DataFrame from the "NEWS_SENTIMENT" function to extract useful data.
#------

def extract_sentiment(df, symbol: str):

    # Conditional to verify that "symbol" is a string:
    if not isinstance(symbol, str):
        raise TypeError(f'The "symbol" parameter must be a string; it\'s currently {str(type(symbol)).upper()}.')

    # Declaration of an empty DataFrames for extracting data and merging before return:
    df_time = pd.DataFrame(columns=['id', 'time_published'])
    df_sentiment = pd.DataFrame(columns=['id', 'relevance_score', 'ticker_sentiment_score', 'ticker_sentiment_label'])

    # Declaring the amount of returned articles to be looped over:
    count = df.shape[0]

    # Loop that pulls the "time_published", "relevance_score", "ticker_sentiment_score", and "ticker_sentiment_label" for the given stock symbol
    # from every row of the "NEWS_SENTIMENT" DataFrame:
    for id in range(count):
        time_row = pd.DataFrame({
            'id': [id],
            'time_published': [df.loc[id, 'feed']['time_published']]
        })
        df_time = pd.concat([df_time, time_row], ignore_index=True)
        for sentiment in df.loc[id, 'feed']['ticker_sentiment']:
            if sentiment['ticker'] == symbol:
                sentiment_row = pd.DataFrame({
                    'id': [id],
                    'relevance_score': [sentiment['relevance_score']],
                    'ticker_sentiment_score':[sentiment['ticker_sentiment_score']],
                    'ticker_sentiment_label': [sentiment['ticker_sentiment_label']]
                })
                df_sentiment = pd.concat([df_sentiment, sentiment_row], ignore_index=True)

    merged_df = df_time.merge(df_sentiment, on='id').drop(columns='id')

    return merged_df

In [10]:
# TimeSeries Indexer
#------
# Applied to every DataFrame produce by an API call.
#------

def set_time_index(df):

    for column in df.columns:
        if str(column).startswith('time') or str(column).endswith('time'):
            df.set_index(pd.to_datetime(df[column]).dt.date, inplace=True)
            df.sort_index(inplace=True)
            df.drop(columns=column, inplace=True)
            df.index.name = 'time'

    return df

In [24]:
# Data Import Function
# ------
# Can create a tuple of DataFrames indexed on a TimeSeries, ready to be processed and merged into a train/test split.
# ------

def alpha_supercall(function_set: str|tuple|list|set, symbol: str, datatype: str, base_query='https://www.alphavantage.co/query?', apikey=os.getenv('ALPHAVANTAGE_API_KEY')):

    # Conditional to verify that "function_set" is either a single string value or iterable of strings:
    if isinstance(function_set, str):
        function_string = function_set
        function_set = []
        function_set.append(function_string)
    elif isinstance(function_set, (int, float, bool)):
        raise TypeError(f'The "function_set" parameter either needs to be a single function passed as a string or an iterable set of strings.\nFUNCTION_SET: {str(type(function_set)).upper()}')
    else:
        if not all(isinstance(item, str) for item in function_set):
            raise TypeError('All items in "function_set" must be string values of the function you\'d like to call.')

    # Conditional to verify that "symbol" and "datatype" are strings:
    if not isinstance(symbol, str) or not isinstance(datatype, str):
        raise TypeError(f'Both the "symbol" and "datatype" parameters must be strings.\nSYMBOL: {str(type(symbol)).upper()}\nDATATYPE: {str(type(datatype)).upper()}')

    # Allow the user to enter a symbol in lowercase without breaking the call:
    symbol = symbol.upper()

    # DataFrame list to be converted to a tuple before being returned to the user:
    dataframes = []

    # Counter for number of calls made to the API in a single function call:
    calls = 0

    for function in function_set:

        # In case a lowercase function is passed:
        function = function.upper()

        # Applies the API key to each function being called:
        parameters[function]['apikey'] = apikey

        # The first set of conditionals that checks the "parameters" dictionary and verifies if "symbol" and "datatype" keys are present
        # within the nested function dictionary, setting them accordingly:
        if 'symbol' and 'datatype' in parameters[function].keys():
            parameters[function]['symbol'] = symbol
            parameters[function]['datatype'] = datatype
        elif 'symbol' not in parameters[function].keys() and 'datatype' in parameters[function].keys():
            parameters[function]['datatype'] = datatype
        else:
            parameters[function]['symbol'] = symbol

        # The second set of conditionals that checks the function type:

        # The "NEWS_SENTIMENT" function only returns JSON with a ton of data that isn't relevant to a prediction model,
        # this conditional statement passes the resulting DataFrame from a "NEWS_SENTIMENT" call
        # through Sentiment Extraction Function:
        if function == 'NEWS_SENTIMENT':
            parameters[function]['tickers'] = symbol
            # Uncomment OPTION 1 and recomment OPTION 2 if you don't want to extract the sentiment and want the pure JSON dictionary:
            # df = pd.DataFrame(requests.get(base_query + urlencode(parameters[function])).json())                              # OPTION 1
            df = extract_sentiment(pd.DataFrame(requests.get(base_query + urlencode(parameters[function])).json()), symbol)     # OPTION 2 (DEFAULT)
            calls += 1
            df = set_time_index(df)
            dataframes.append(df)
            print(f'> "{function}" DataFrame created!')
        else:
            df = pd.read_csv(StringIO(requests.get(base_query + urlencode(parameters[function])).text))
            calls += 1
            df = set_time_index(df)
            dataframes.append(df)
            print(f'> "{function}" DataFrame created!')

    global session_calls
    session_calls += calls

    # Prints the amount of API calls made in the function call and the total calls made in the current session:
    print(f'\nCALL WEIGHT: {calls}')
    print(f'TOTAL CALLS MADE: {session_calls}')

    # Returns a single DataFrame if only one function is passed:
    if len(function_set) == 1:
        return dataframes[0]

    # Returns a tuple to unpack into multiple DataFrames if multiple functions are passed:
    else:
        return tuple(dataframes)

## First Set

In [29]:
aapl_tsd, aapl_rsi, aapl_ns, aapl_macd = alpha_supercall(function_set, 'AAPL', 'csv')

> "TIME_SERIES_DAILY" DataFrame created!
> "RSI" DataFrame created!
> "NEWS_SENTIMENT" DataFrame created!
> "MACD" DataFrame created!

CALL WEIGHT: 4
TOTAL CALLS MADE: 37


In [ ]:
aapl_tsd.head()

,open,high,low,close,volume
time,,,,,
1999-11-01,80.00,80.69,77.37,77.62,2487300
1999-11-02,78.00,81.69,77.31,80.25,3564600
1999-11-03,81.62,83.25,81.00,81.50,2932700
1999-11-04,82.06,85.37,80.62,83.62,3384700
1999-11-05,84.62,88.37,84.00,88.31,3721500


In [ ]:
aapl_tsd.shape

(6141, 5)

In [ ]:
aapl_rsi.head(10)

,RSI
time,
1999-11-19,69.9838
1999-11-22,66.4703
1999-11-23,68.5184
1999-11-24,70.2012
1999-11-26,70.5350
1999-11-29,69.4036
1999-11-30,72.5435
1999-12-01,76.5988
1999-12-02,80.7953


In [ ]:
aapl_rsi.shape

(6127, 1)

In [ ]:
aapl_macd.head()

,MACD,MACD_Hist,MACD_Signal
time,,,
1999-12-17,0.0243,-0.0174,0.0417
1999-12-20,0.0211,-0.0165,0.0376
1999-12-21,0.0211,-0.0132,0.0343
1999-12-22,0.0193,-0.0120,0.0313
1999-12-23,0.0198,-0.0092,0.0290


In [ ]:
aapl_ns.shape

(685, 3)

In [ ]:
aapl = aapl_tsd.merge(aapl_rsi, left_index=True, right_index=True) \
    .merge(aapl_macd, left_index=True, right_index=True)

In [ ]:
aapl = generate_features(aapl)

In [ ]:
aapl.head()

,open,high,low,close,lag_1,lag_2,rolling_mean_7,rolling_std_7,daily_return,volume,RSI,MACD,MACD_Hist,MACD_Signal
time,,,,,,,,,,,,,,
1999-12-28,99.12,99.62,95.00,98.19,99.31,103.50,100.205714,2.080047,-1.127782,2210500,50.5363,0.0147,-0.0096,0.0243
1999-12-29,96.81,102.19,95.50,100.69,98.19,99.31,100.304286,2.085017,2.546084,2540200,53.5448,0.0139,-0.0083,0.0222
1999-12-30,102.19,104.12,99.62,100.31,100.69,98.19,100.634286,1.826279,-0.377396,1849500,53.0170,0.0129,-0.0074,0.0203
1999-12-31,100.94,102.87,99.50,102.81,100.31,100.69,100.678571,1.881971,2.492274,1462600,56.0843,0.0135,-0.0055,0.0190
2000-01-03,104.87,112.50,101.69,111.94,102.81,100.31,102.392857,4.599887,8.880459,4783900,65.0564,0.0193,0.0003,0.0190


In [ ]:
aapl.shape

(6102, 14)

In [ ]:
# export_csv(aapl, 'aapl')

## Second Set

In [ ]:
msft_tsd, msft_rsi, msft_ns, msft_macd = alpha_supercall(function_set, 'msft', 'csv')

> "TIME_SERIES_DAILY" DataFrame created!
> "RSI" DataFrame created!
> "NEWS_SENTIMENT" DataFrame created!
> "MACD" DataFrame created!

CALL COUNT: 4


In [ ]:
msft_tsd.head()

,open,high,low,close,volume
time,,,,,
1999-11-01,93.25,94.19,92.12,92.37,26630600
1999-11-02,92.75,94.50,91.94,92.56,23174500
1999-11-03,92.94,93.50,91.50,92.00,22258500
1999-11-04,92.31,92.75,90.31,91.75,27119700
1999-11-05,91.81,92.87,90.50,91.56,35083700


In [ ]:
msft = msft_tsd.merge(msft_rsi, left_index=True, right_index=True) \
    .merge(msft_macd, left_index=True, right_index=True)

In [ ]:
msft = generate_features(msft)

In [ ]:
msft.head()

,open,high,low,close,lag_1,lag_2,rolling_mean_7,rolling_std_7,daily_return,volume,RSI,MACD,MACD_Hist,MACD_Signal
time,,,,,,,,,,,,,,
1999-12-28,118.75,118.81,117.06,117.50,119.12,117.44,116.498571,2.077398,-1.359973,12295500,75.2622,2.3195,0.5222,1.7973
1999-12-29,116.94,118.37,116.81,117.94,117.50,119.12,116.882857,2.056637,0.374468,8724600,75.6687,2.3114,0.4113,1.9001
1999-12-30,117.87,119.94,117.12,117.62,117.94,117.50,117.578571,0.953352,-0.271324,11180000,74.7072,2.2707,0.2965,1.9742
1999-12-31,117.50,117.75,116.25,116.75,117.62,117.94,117.704286,0.719950,-0.739670,6258800,72.0276,2.1915,0.1738,2.0177
2000-01-03,117.37,118.62,112.00,116.56,116.75,117.62,117.561429,0.842188,-0.162741,26614200,71.4250,2.0997,0.0657,2.0341


In [ ]:
msft.shape

(6102, 14)

In [ ]:
# export_csv(msft, 'msft')